In order to use this example, you must first have deployed the Diet example.
A REST API is provided to submit input data, solve, and get results.
Its use is shown below.

In [ ]:
import requests 
import json
import pandas
import os

server = "ibm-nginx-svc"

urlAlias = "arno"
deployedAssetName = "diet"
savedModelName = "savedDiet"

executionServiceURL = 'https://' + server + "/dsvc/v1/" + urlAlias + "/domodel/" + deployedAssetName
executionServiceModelURL = executionServiceURL + "/model/" + savedModelName

In [ ]:
executionToken = "<Here paste the token you can find in IBM Watson Machine Learning>"
headers = {"Authorization" : executionToken}

In [ ]:
# GET Solve configuration 
r = requests.get(executionServiceModelURL,  headers=headers)
obj = r.json();
SOLVE_URL = [x['uri'] for x in obj['deploymentDescription']['links'] if x['target'] == 'solve'][0]
SOLVE_CONFIG = obj['deploymentDescription']['solveConfig']


# Debug
If needed, you can export input/output tables and the model for a given job.  
This can be done either:
  * in the deployment:
The run configuration is now part of the saved model.  
If you activate the dump of debug material by specifying `oaas.dumpZipName`.  
You can also specify some rules that trigger the dump. That can be done using `oaas.dumpZipRules`.  
The dump zip file is generated in this specific container: `/data/containers/debug` 
  * in Job solve configuration:  
As follows.

In [ ]:
SOLVE_CONFIG['solveParameters'] = {"oaas.dumpZipName": ""}
SOLVE_CONFIG['attachments'].append({'category': 'output', 'type': 'CONTAINER', 'containerId': 'debug', 'name' : '.*\\.zip'})


# Prepare the data for the solve

In [ ]:
solve_data = {}
diet_food_columns=["name","unit_cost","qmin","qmax"]
diet_food_data=[
    ("Roasted Chicken", 0.84, 0, 10),
    ("Spaghetti W/ Sauce", 0.78, 0, 10),
    ("Tomato,Red,Ripe,Raw", 0.27, 0, 10),
    ("Apple,Raw,W/Skin", .24, 0, 10),
    ("Grapes", 0.32, 0, 10),
    ("Chocolate Chip Cookies", 0.03, 0, 10),
    ("Lowfat Milk", 0.23, 0, 10),
    ("Raisin Brn", 0.34, 0, 10),
    ("Hotdog", 0.31, 0, 10)]
solve_data['diet_food'] = pandas.DataFrame.from_records(diet_food_data,columns=diet_food_columns)

diet_nutrients_columns=["name","qmin","qmax"]
diet_nutrients_data=[
    ("Calories", 2000, 2500),
    ("Calcium", 800, 1600),
    ("Iron", 10, 30),
    ("Vit_A", 5000, 50000),
    ("Dietary_Fiber", 25, 100),
    ("Carbohydrates", 0, 300),
    ("Protein", 50, 100)]
solve_data['diet_nutrients'] = pandas.DataFrame.from_records(diet_nutrients_data,columns=diet_nutrients_columns)

diet_food_nutrients_columns=["Food","Calories","Calcium","Iron","Vit_A","Dietary_Fiber","Carbohydrates","Protein"]
diet_food_nutrients_data=[
    ("Roasted Chicken", 277.4, 21.9, 1.8, 77.4, 0, 0, 42.2),
    ("Spaghetti W/ Sauce", 358.2, 80.2, 2.3, 3055.2, 11.6, 58.3, 8.2),
    ("Tomato,Red,Ripe,Raw", 25.8, 6.2, 0.6, 766.3, 1.4, 5.7, 1),
    ("Apple,Raw,W/Skin", 81.4, 9.7, 0.2, 73.1, 3.7, 21, 0.3),
    ("Grapes", 15.1, 3.4, 0.1, 24, 0.2, 4.1, 0.2),
    ("Chocolate Chip Cookies", 78.1, 6.2, 0.4, 101.8, 0, 9.3, 0.9),
    ("Lowfat Milk", 121.2, 296.7, 0.1, 500.2, 0, 11.7, 8.1),
    ("Raisin Brn", 115.1, 12.9, 16.8, 1250.2, 4, 27.9, 4),
    ("Hotdog", 242.1, 23.5, 2.3, 0, 0, 18, 10.4)]
solve_data['diet_food_nutrients'] = pandas.DataFrame.from_records(diet_food_nutrients_data,columns=diet_food_nutrients_columns)
 
files = {'solveconfig': json.dumps(SOLVE_CONFIG)};
for i in solve_data:
    files[i+'.csv'] = solve_data[i].to_csv();

# Execution

In [ ]:
# Send the job
r = requests.post(SOLVE_URL, files=files, headers=headers)
obj = r.json();

JOB_URL = [x['href'] for x in obj['links'] if x['rel'] == 'self'][0]

# Pool for Job status
from time import sleep

while True:
    sleep(1)
    r = requests.get(JOB_URL, headers=headers)
    executionStatus = r.json();
    status = executionStatus['solveState']['executionStatus']
    print(status)
    if status == "PROCESSED":
        for o in executionStatus['outputAttachments']:
            print(o['name'] + '  ' + o['url'])
        break
    if status == "FAILED":
        print("Failure message: " + executionStatus['solveState']['failureInfo']['message'])
        break
    if status == "INTERRUPTED":
        print("Interruption status: " + executionStatus['solveState']['interruptionStatus'])
        break
        

# Retrieve debug material
In this example, the dump zip file is saved in notebook folder.  
This dump zip file can be used to create a new scenario (Using the right-hand-side panel in the Model Builder, 'Create Scenario From file')


In [ ]:
for o in executionStatus['outputAttachments']:
    if "dump_" in o['name']:
        zipName = o['name']
        DEBUG_FILE =  o['url']
        print(DEBUG_FILE)
        z = requests.get(DEBUG_FILE, headers=headers,stream=True)
        f = open(zipName, 'wb')
        for chunk in z.iter_content(chunk_size=512 * 1024):
            if chunk:
                f.write(chunk)
            f.close()

# Cleanup
Need to delete the job from the execution service as well as the debug file from the "debug" container, to free space on the cluster


In [ ]:
requests.delete(JOB_URL, headers=headers)
requests.delete(executionServiceURL+'/data/containers/debug/'+zipName, headers=headers)
